# Grayscale and normalize images
This notebook converts images from input folder to grayscale and normalizes intensity, shipping them to an output folder.

In the paper, this notebook was used as the step between FIJI and Adobe Photoshop for cross section images. The workflow was as follows: 
- FIJI: Use BioImporter to select the 3D .ims file of an rhTestis. 
- FIJI: Select a single Z slice to obtain a cross section view. 
- FIJI: Save the Z slice as separate .tiff files per channel. 

- Then, go to this notebook. Here, the .tiff files will be turned into grayscale images that will be normalized for minimum and maximum signal. The output .tiff files can then be parsed into Photoshop as one image. 


Of note: 
- In your directory where this .ipynb notebook is stored, please make a folder and name this "RawImages" -> put your .tif files in this "Images" folder. 
- Do not create an output folder, the code will do this. 
- This notebook will process all the images in your input folder. 
- The code assumes 16-bit images, but you may change this if needed in 'def normalize_data'.  


First, we import the libraries we are going to use:

In [1]:
# standard lib
from pathlib import Path # only import Path, not able to use the rest of pathlib
import os
import shutil

# 3rd party
import tifffile as tiff
import numpy as np

Get current working directory (the folder you are working from) and assign it to the variable root folder. Next, specify input and output folder.

In [2]:
ROOT_FOLDER = Path(os.getcwd()) / "YOURFOLDER"

INPUT_FOLDER = ROOT_FOLDER / "RawImages"
OUTPUT_FOLDER = ROOT_FOLDER / "NormalizedImages"

Now we are going to define a function for normalizing image data. We set the lowest intensity to be desired_min (default 0) and the highest intensity to be desired_max (default `2**16` for 16 bit images). Scales everything in between linearly.

In [3]:
def normalize_data(data, desired_min=0, desired_max=2**16):
    norm_data = (data - np.min(data)) / (np.max(data) - np.min(data)) # converts data to be between 0 (lowest) and 1 (highest)
    return (norm_data * (desired_max - desired_min)) + desired_min # converts 0-1 interval to desired_min, desired_max interval

# Uses:
# normalize_data(data) from 0 to 2**16 (16 bit)
# normalize_data(data, desired_max=2**8) (8 bit)

Next, define a function for converting an input file (path to file) to an output file (save image to output file). It converts the image to grayscale (actually the image is already grayscale, but it has no color space, saving it solves this issue). It also normalizes the data, using the function above.

In [4]:
def convert_to_grayscale_and_normalize(input_file, output_file):
    img = tiff.imread(input_file) # read the file using tiff (tifffile)
    normalized_img = normalize_data(img).astype("uint16") # 16 bit integer (unsigned, i.e. only positive)
    tiff.imsave(output_file, normalized_img) # Save normalized_img to output_file

Copy import folder (and all subfolders) to output folder (keeps the structure as well). We do this because we do not want to overwrite the original images, so we first copy and then normalize/convert.

In [ ]:
# If the following throws an error, the output folder probably already exists. Try removing it and run again.
shutil.copytree(INPUT_FOLDER, OUTPUT_FOLDER)

Now we get all the files in the output folder (so the copied files) and for each file, convert to grayscale and normalize the image.
This overwrites the file in the output folder.

In [ ]:
# rglob is a way to traverse the directory recursively (get all the files of all the folders in the directory)
# The file filter used (*.tif) makes sure we only get .tif files (only TIFF images) and all of them
for filename in OUTPUT_FOLDER.rglob('*.tif'):
    convert_to_grayscale_and_normalize(filename, filename) # just overwrite, we already copied them